# 1. Assignment setup

In [6]:
cd sars2copath/analyses

In [7]:
ls

blastn		     outputs
blastn_para	     R_studio_abundances_heatmaps.nb.html
bowtie		     R_studio_abundances_heatmaps.Rmd
data		     sample_collab.db
Day1afternoon.ipynb  sample_collab_db_schema.sql
history.txt	     Untitled.ipynb
kraken2		     zliu_flash2.log
krona		     zliu_run_accessions.txt


In [8]:
cd ../scripts

In [9]:
ls

bash_create_sample_collab_sqlite_db.ipynb  kraken2_sbatch_array.sh
blastn_parallel.sh			   kreport2krona_sbatch_array_1.sh
blastn_sbatch_array.sh			   kreport2krona_sbatch_array.sh
generic_sbatch_array.sh			   outputs
kraken2_bracken_sbatch_array.sh		   ziwei_read_QC.sh


chmod a+x ziwei_read_QC.sh 

### To execute your script, just type:

# 2. Programmatic access to NCBI SRA

###Firstly, export your list of sequencing run identifiers in a file which we will use to download the corresponding FASTQ files using sqlite3:

In [2]:
sqlite3 -batch /shared/projects/form_2022_19/pascal/central_database/sample_collab.db \
"select run_accession from (select spl.run_accession from sample_annot spl left join sample2bioinformatician s2b using(patient_code) where username='zliu');" -noheader -csv > zliu_run_accessions.text 

In [5]:
cd sars2copath/analyses
ls

bash: cd: sars2copath/analyses: No such file or directory
blastn		     outputs
blastn_para	     R_studio_abundances_heatmaps.nb.html
bowtie		     R_studio_abundances_heatmaps.Rmd
data		     sample_collab.db
Day1afternoon.ipynb  sample_collab_db_schema.sql
history.txt	     Untitled.ipynb
kraken2		     zliu_flash2.log
krona		     zliu_run_accessions.txt


In [2]:
cd sars2copath/data

In [3]:
ls #mkdir sra_fastq -- create a new directory to store raw fastq file form NCBI 

blast_db      blast_db.nto			   sample_overviews 3.csv
blast_db.ndb  bowtie2_DBs			   sample_overviews 5.csv
blast_db.nhr  ct_values_covid19_metagenomics.xlsx  slurm.23319766.1.er
blast_db.nin  filereport_read_run_PRJEB47870.tsv   sra_fastq
blast_db.not  merged_pairs			   SraRunTable.csv
blast_db.nsq  para_computing			   subset
blast_db.ntf  reference_seqs


To download all 8 sequencing run ID
Use xargs to convert IDs into a concatenation of 8 arguments which added at the end of the fastq-dump command 
because the usual invocation of xargs involves a "pipe" (as in cat identifiers.txt | xargs fastq-dump) you have to be careful not to include the pipe after srun, so you can use either one of the two following solutions:

In [7]:
cd sra_fastq

In [8]:
ls

ERR6913297_1.fastq.gz  ERR6913300_1.fastq.gz  ERR6913328_1.fastq.gz
ERR6913297_2.fastq.gz  ERR6913300_2.fastq.gz  ERR6913328_2.fastq.gz
ERR6913298_1.fastq.gz  ERR6913326_1.fastq.gz  ERR6913329_1.fastq.gz
ERR6913298_2.fastq.gz  ERR6913326_2.fastq.gz  ERR6913329_2.fastq.gz
ERR6913299_1.fastq.gz  ERR6913327_1.fastq.gz  fastqc
ERR6913299_2.fastq.gz  ERR6913327_2.fastq.gz


In [9]:
wc -l ERR6913297_1.fastq.gz
# If you’re absolutely positive your FASTQ file uses four lines per sequence entry, you can estimate the number of sequences by estimating the number of lines with
# and dividing by four

### Manipulating raw sequencing FASTQ file with seqkit

In [1]:
module load seqkit

In [2]:
seqkit -h

SeqKit -- a cross-platform and ultrafast toolkit for FASTA/Q file manipulation

Version: 2.1.0

Author: Wei Shen <shenwei356@gmail.com>

Documents  : http://bioinf.shenwei.me/seqkit
Source code: https://github.com/shenwei356/seqkit
Please cite: https://doi.org/10.1371/journal.pone.0163962


Seqkit utlizies the pgzip (https://github.com/klauspost/pgzip) package to
read and write gzip file, and the outputted gzip file would be slighty
larger than files generated by GNU gzip.

Seqkit writes gzip files very fast, much faster than the multi-threaded pigz,
therefore there's no need to pipe the result to gzip/pigz.

Usage:
  seqkit [command]

Available Commands:
  amplicon        extract amplicon (or specific region around it) via primer(s)
  bam             monitoring and online histograms of BAM record features
  common          find common sequences of multiple files by id/name/sequence
  concat          concatenate sequences with same ID from multiple files
  convert         convert FASTQ

### Quality control the raw sequencing FASTQ files with fastQC

In [2]:
module load fastqc

Use srun and a couple of threads

In [8]:
ls

ERR6913297_1_fastqc.html  ERR6913299_2_fastqc.zip   ERR6913327_2_fastqc.html
ERR6913297_1_fastqc.zip   ERR6913300_1_fastqc.html  ERR6913327_2_fastqc.zip
ERR6913297_2_fastqc.html  ERR6913300_1_fastqc.zip   ERR6913328_1_fastqc.html
ERR6913297_2_fastqc.zip   ERR6913300_2_fastqc.html  ERR6913328_1_fastqc.zip
ERR6913298_1_fastqc.html  ERR6913300_2_fastqc.zip   ERR6913328_2_fastqc.html
ERR6913298_1_fastqc.zip   ERR6913326_1_fastqc.html  ERR6913328_2_fastqc.zip
ERR6913298_2_fastqc.html  ERR6913326_1_fastqc.zip   ERR6913329_1_fastqc.html
ERR6913298_2_fastqc.zip   ERR6913326_2_fastqc.html  ERR6913329_1_fastqc.zip
ERR6913299_1_fastqc.html  ERR6913326_2_fastqc.zip   ERR6913329_2_fastqc.html
ERR6913299_1_fastqc.zip   ERR6913327_1_fastqc.html  ERR6913329_2_fastqc.zip
ERR6913299_2_fastqc.html  ERR6913327_1_fastqc.zip


### Moving files from remote server to local laptop hard disk

### Adapator and low quality read trimming

### Merging paired end reads

In [7]:
ls

blastn		     outputs
blastn_para	     R_studio_abundances_heatmaps.nb.html
bowtie		     R_studio_abundances_heatmaps.Rmd
data		     sample_collab.db
Day1afternoon.ipynb  sample_collab_db_schema.sql
history.txt	     Untitled.ipynb
kraken2		     zliu_flash2.log
krona		     zliu_run_accessions.txt


Set cpus to 2 to match srun reservation
Set the output files with .flash
Set the output files to be compressed
Capture the program output by adding 2>&1 | tee your_username_flash2.log
this allows you to follow the execution output live on your screen, whilst at the same time recording the very same output to a log file for future reference (in fact this log file will be used later by another  tool called MultiQ)
the 2>&1 notation says that both STANDARD OUTPUT and STANDARD ERROR streams are recorded in the log file

In [19]:
ls

ERR6913297.flash.extendedFrags.fastq.gz
ERR6913297.flash.hist
ERR6913297.flash.histogram
ERR6913297.flash.notCombined_1.fastq.gz
ERR6913297.flash.notCombined_2.fastq.gz
ERR6913298.flash.extendedFrags.fastq.gz
ERR6913298.flash.hist
ERR6913298.flash.histogram
ERR6913298.flash.notCombined_1.fastq.gz
ERR6913298.flash.notCombined_2.fastq.gz
ERR6913299.flash.extendedFrags.fastq.gz
ERR6913299.flash.hist
ERR6913299.flash.histogram
ERR6913299.flash.notCombined_1.fastq.gz
ERR6913299.flash.notCombined_2.fastq.gz
ERR6913300.flash.extendedFrags.fastq.gz
ERR6913300.flash.hist
ERR6913300.flash.histogram
ERR6913300.flash.notCombined_1.fastq.gz
ERR6913300.flash.notCombined_2.fastq.gz
ERR6913326.flash.extendedFrags.fastq.gz
ERR6913326.flash.hist
ERR6913326.flash.histogram
ERR6913326.flash.notCombined_1.fastq.gz
ERR6913326.flash.notCombined_2.fastq.gz
ERR6913327.flash.extendedFrags.fastq.gz
ERR6913327.flash.hist
ERR6913327.flash.histogram
ERR6913327.flash.notCombined_1.fastq.gz
ERR6913327.flash.notCombin

### Use read mapping to check for PhiX contamination 

In [22]:
cd reference_seqs
ls

PhiX_NC_001422.fna


It is customary for fast sequence alignment software (also BLAST, MMSEQS2 etc.), 
you first need to create a bowtie2 indexed database from the reference sequences you wish to align the reads to (adapt as necessary for your sequence file names

In [26]:
cd ../bowtie2_DBs
ls

PhiX_bowtie2_DB.1.bt2  PhiX_bowtie2_DB.3.bt2  PhiX_bowtie2_DB.rev.1.bt2
PhiX_bowtie2_DB.2.bt2  PhiX_bowtie2_DB.4.bt2  PhiX_bowtie2_DB.rev.2.bt2


In [27]:
cd ../../analyses

In [30]:
cd bowtie
ls

zliu_bowtie_merged2PhiX.log  zliu_merged2PhiX.sam


In [44]:
cd bowtie2_DBs
ls

PhiX_bowtie2_DB.1.bt2  PhiX_bowtie2_DB.rev.1.bt2  SC2_bowtie2_DB.3.bt2
PhiX_bowtie2_DB.2.bt2  PhiX_bowtie2_DB.rev.2.bt2  SC2_bowtie2_DB.4.bt2
PhiX_bowtie2_DB.3.bt2  SC2_bowtie2_DB.1.bt2	  SC2_bowtie2_DB.rev.1.bt2
PhiX_bowtie2_DB.4.bt2  SC2_bowtie2_DB.2.bt2	  SC2_bowtie2_DB.rev.2.bt2


### Combine quality control results into one unique report for all samples analyses

In [47]:
module load multiqc

In [48]:
multiqc --help

Usage: multiqc [OPTIONS] <analysis directory>

  MultiQC aggregates results from bioinformatics analyses across many samples
  into a single report.

  It searches a given directory for analysis logs and compiles a HTML report.
  It's a general use tool, perfect for summarising the output from numerous
  bioinformatics tools.

  To run, supply with one or more directory to scan for analysis results. To
  run here, use 'multiqc .'

  See http://multiqc.info for more details.

  Author: Phil Ewels (http://phil.ewels.co.uk)

Options:
  -f, --force                     Overwrite any existing reports
  -d, --dirs                      Prepend directory to sample names
  -dd, --dirs-depth INTEGER       Prepend [INT] directories to sample names.
                                  Negative number to take from start of path.
  -s, --fullnames                 Do not clean the sample names (leave as full
                                  file name)
  -i, --title TEXT                Report title. Pri

-- multiqc automagically guesses where all these files came from and digests their contents
adding at least the --title to individualise all participant reports
add the --force switch so you don't get incremental reports each time you run this command

check out your SLURM usage so far